# Zadanie
Przy użyciu pakietów numpy i pandas odtwórz w Pythonie działanie zapytań SQL.
Using packages numpy and pandas emulate effects of given SQL queries.

# 1. Import pakietów

In [2]:
import numpy as np
import pandas as pd

# 2. Ładowanie danych

## 2.1 Tworzenie bazy danych

In [3]:
import tempfile
import os
baza = os.path.join(tempfile.mkdtemp(), "baza.db")

if os.path.isfile(baza):
    os.remove(baza)

import sqlite3
conn = sqlite3.connect(baza)

## 2.2 Ładowanie zbiorów do zmiennych

In [4]:
Posts = pd.read_csv('Posts.csv.gz',
                      compression = 'gzip',
                      error_bad_lines = False,
                      comment = "#")
Users = pd.read_csv('Users.csv.gz',
                      compression = 'gzip',
                      error_bad_lines = False,
                      comment = "#")
Votes = pd.read_csv('Votes.csv.gz',
                      compression = 'gzip',
                      error_bad_lines = False,
                      comment = "#")
Comments = pd.read_csv('Comments.csv.gz',
                      compression = 'gzip',
                      error_bad_lines = False,
                      comment = "#")
Badges = pd.read_csv('Badges.csv.gz',
                      compression = 'gzip',
                      error_bad_lines = False,
                      comment = "#")



## 2.3 Ładowanie danych do bazych danych

In [5]:
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)
Comments.to_sql("Comments", conn)
Badges.to_sql("Badges", conn)

# 3. Zapytania

## Zadanie 1

### Zapytanie

In [6]:
pd.read_sql_query('''SELECT
        Users.DisplayName,
        Users.Age,
        Users.Location,
        SUM(Posts.FavoriteCount) AS FavoriteTotal,
        Posts.Title AS MostFavoriteQuestion,
        MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
      FROM Posts
      JOIN Users ON Users.Id=Posts.OwnerUserId
      WHERE Posts.PostTypeId=1
      GROUP BY OwnerUserId
      ORDER BY FavoriteTotal DESC
      LIMIT 10''', conn)

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


### Implementacja

In [7]:
x1 = Posts
x2 = Users

x2 = x2.rename(columns = {'Id':'OwnerUserId'})
x = pd.merge(x1, x2, how = 'inner', on = 'OwnerUserId')
x = x[x.PostTypeId == 1]

x = x[['DisplayName', 'Age', 'Location', 'FavoriteCount', 'Title', 'OwnerUserId']]

tmp = x.groupby(by = "OwnerUserId")

df1 = pd.DataFrame(tmp['FavoriteCount'].max())
df1 = df1.rename(columns = {'FavoriteCount':'MostFavoriteQuestionLikes'})
df2 = pd.DataFrame(tmp['FavoriteCount'].sum())
df2 = df2.rename(columns = {'FavoriteCount':'FavoriteTotal'})


x = pd.merge(x, df1, how = 'inner', on = 'OwnerUserId')
x = pd.merge(x, df2, how = 'inner', on = 'OwnerUserId')
x = x[x.MostFavoriteQuestionLikes == x.FavoriteCount]
tmp2 = pd.DataFrame(x.DisplayName.drop_duplicates())
x = pd.merge(tmp2, x, how = 'inner', on = 'DisplayName')
x = x.sort_values('FavoriteTotal', ascending = False)
x = x.drop(columns = ['FavoriteCount', 'OwnerUserId'])
x = x.rename(columns = {'Title':'MostFavoriteQuestion'})
x = x.reset_index(drop = True)
x[0:10]

,DisplayName,Age,Location,MostFavoriteQuestion,MostFavoriteQuestionLikes,FavoriteTotal
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",Tactics to avoid getting harassed by corrupt p...,42.0,467.0
1,hippietrail,NaN,"Oaxaca, Mexico","OK we're all adults here, so really, how on ea...",79.0,444.0
2,RoflcoptrException,NaN,NaN,How to avoid drinking vodka?,29.0,294.0
3,JonathanReez,26.0,"Prague, Czech Republic",What is the highest viewing spot in London tha...,17.0,221.0
4,nsn,NaN,NaN,How do airlines determine ticket prices?,40.0,214.0
5,Gagravarr,NaN,"Oxford, United Kingdom",Are there other places with gardens like those...,10.0,151.0
6,Andrew Grimm,38.0,"Sydney, Australia","OK we're all nerds here, so really, how on ear...",8.0,120.0
7,VMAtm,33.0,"Tampa, FL, United States",Is there a good website to plan a trip via tra...,34.0,109.0
8,jrdioko,NaN,NaN,What is the most comfortable way to sleep on a...,21.0,100.0
9,Gayot Fow,NaN,"London, United Kingdom",Should I submit bank statements when applying ...,18.0,98.0


## Zadanie 2

### Zapytanie

In [8]:
pd.read_sql_query('''SELECT
Posts.ID,
Posts.Title,
Posts2.PositiveAnswerCount
FROM Posts
JOIN (
SELECT
Posts.ParentID,
COUNT(*) AS PositiveAnswerCount
FROM Posts
WHERE Posts.PostTypeID=2 AND Posts.Score>0
GROUP BY Posts.ParentID
) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10''', conn)


,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


### Implementacja

In [9]:
x1 = Posts
x2 = Posts #2

x2 = x2[x2.PostTypeId == 2]
x2 = x2[x2.Score > 0]
x2 = x2.groupby(by = 'ParentId')
x2 = x2.size().reset_index()
x2.columns = ['Id', 'PositiveAnswerCount']

x = pd.merge(x1, x2, how = 'inner', on = 'Id')
x = x[['Id', 'Title', 'PositiveAnswerCount']]


x = x.sort_values('PositiveAnswerCount', ascending = False)
x = x.reset_index(drop = True)
x[0:10]

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,7663,Am I expected to tip wait staff in Europe?,17
9,59128,"Is there a way to prevent ""looking like a tour...",17


## Zadanie 3

### Zapytanie

In [10]:
pd.read_sql_query('''SELECT
Posts.Title,
UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
SELECT
PostId,
COUNT(*) AS Count,
STRFTIME('%Y', Votes.CreationDate) AS Year
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, Year
) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year''', conn)

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


### Implementacja

In [11]:
x1 = Posts
x2 = Votes

def get_year(q):
    return int(q[0:4])

x2 = x2[x2.VoteTypeId == 2]
x2['Year'] = x2.CreationDate.apply(get_year)

x2 = x2.groupby(by = ['PostId', 'Year'])
x2 = x2.size().reset_index()
x2.columns = ['Id', 'Year', 'Count'] #UpVotesPerYear
x = pd.merge(x1, x2, how = 'inner', on = 'Id')
x = x[x.PostTypeId == 1]

tmp = x.groupby(by = 'Year')
tmp = tmp['Count'].max().reset_index()
x = pd.merge(tmp, x, how = 'inner', on = ['Year', 'Count'])
x = x[['Title', 'Year', 'Count']]
x

c:\users\staro\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


## Zadanie 4

### Zapytanie

In [12]:
pd.read_sql_query('''SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT *
FROM Posts
WHERE PostTypeId==1
) AS Questions ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC''', conn)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


### Implementacja

In [13]:
x1 = Posts
x2 = Posts
x3 = Posts

x1 = x1[x1.PostTypeId == 2]
tmp = x1.groupby(by = 'ParentId')
tmp = tmp['Score'].max().reset_index()
x1 = pd.merge(x1, tmp, how = 'inner', on = ['ParentId', 'Score'])
x1 = x1[['ParentId', 'Score']]
x1.columns = ['Id', 'MaxScore'] #BestAnswers


x2 = x2[x2.PostTypeId == 1] 
x2 = x2[['Id', 'AcceptedAnswerId', 'Title']]  #Questions


x3 = x3[['Id', 'Score']]
x3.columns = ['AcceptedAnswerId', 'AcceptedScore'] #Posts

x = pd.merge(x1, x2, how = 'inner', on = 'Id')
x = pd.merge(x, x3, how = 'inner', on = 'AcceptedAnswerId')
x['Difference'] = x.MaxScore - x.AcceptedScore
x = x[x.Difference > 50]
x = x.drop(columns = ['AcceptedAnswerId'])
x = x.sort_values('Difference', ascending = False)
x = x.reset_index(drop = True)
x

,Id,MaxScore,Title,AcceptedScore,Difference
0,99713.0,126,"In Germany, what are you supposed to do if you...",33,93
1,81376.0,120,What is way to eat rice with hands in front of...,30,90
2,76737.0,118,Why do many countries in the world still requi...,31,87
3,38177.0,245,How do you know if Americans genuinely/literal...,166,79
4,90636.0,101,Caught speeding 111 Mph (179 km/h) in Californ...,25,76
5,81492.0,90,Why is international first class much more exp...,21,69
6,94320.0,71,Strange looking region of France on Google Maps,7,64
7,80050.0,72,What's an easy way of making my luggage unique...,16,56


## Zadanie 5

### Zapytanie

In [14]:
pd.read_sql_query('''SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10''', conn)

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


### Implementacja

In [15]:
x1 = Comments
x2 = Posts

x1 = x1.groupby(by = ['PostId', 'UserId'])
x1 = x1['Score'].sum().reset_index()
x1.columns = ['Id', 'OwnerUserId', 'CommentsTotalScore']

x = pd.merge(x1, x2, how = 'inner', on = ['Id', 'OwnerUserId'])
x = x[x.PostTypeId == 1]
x = x[['Title', 'CommentsTotalScore']]
x = x.sort_values('CommentsTotalScore', ascending = False)
x = x.reset_index(drop = True)
x[0:10]

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


## Zadanie 6

### Zapytanie

In [16]:
pd.read_sql_query('''SELECT DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT Name, UserID
FROM Badges
WHERE Name IN (
SELECT Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id''', conn)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360.0,NaN,Down underer
4,793,mindcorrosive,10531.0,32.0,Bulgaria
5,693,RoflcoptrException,33300.0,NaN,None
6,6669,Relaxed,69405.0,NaN,None
7,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
8,39065,Pont,1004.0,NaN,Austria
9,19400,phoog,34342.0,50.0,"New York, NY"


### Implementacja

In [17]:
x1 = Badges 

x1 = x1[x1.Class == 1]
x1 = x1.groupby(by = 'Name')
x1 = x1.size().reset_index()
x1.columns = ['Name', 'Count']
x1 = x1[x1.Count > 1]
x1 = x1[x1.Count < 11]

x = pd.merge(x1, Badges, how = 'inner', on = 'Name')
x = x[x.Class == 1]
x = x[['Name', 'UserId']]
x = x.rename(columns = {'UserId':'Id'})
x = pd.merge(x, Users, how = 'inner', on = 'Id')
x = x[['Id', 'DisplayName', 'Reputation', 'Age', 'Location']]
x = x.drop_duplicates().reset_index()
x

,index,Id,DisplayName,Reputation,Age,Location
0,0,108,Ankur Banerjee,31273.0,27.0,"London, UK"
1,2,19,VMAtm,18556.0,33.0,"Tampa, FL, United States"
2,3,101,Mark Mayo,121667.0,37.0,"Sydney, New South Wales, Australia"
3,5,466,iHaveacomputer,8360.0,NaN,Down underer
4,6,39065,Pont,1004.0,NaN,Austria
5,7,793,mindcorrosive,10531.0,32.0,Bulgaria
6,8,693,RoflcoptrException,33300.0,NaN,NaN
7,9,6669,Relaxed,69405.0,NaN,NaN
8,10,1737,Gayot Fow,70237.0,NaN,"London, United Kingdom"
9,11,19400,phoog,34342.0,50.0,"New York, NY"


## Zadanie 7

### Zapytanie

In [18]:
pd.read_sql_query('''SELECT
Posts.Title,
VotesByAge2.OldVotes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2017' THEN 'new'
WHEN '2016' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, VoteDate
) AS VotesByAge
GROUP BY VotesByAge.PostId
HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10''', conn)

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


### Implementacja

In [19]:
x1 = Votes

def set_date(q):
    if int(q[0:4]) > 2015 : return 1
    else : return 0

x1 = x1[x1.VoteTypeId == 2]
x1['VoteDate'] = x1['CreationDate'].apply(set_date)

tmp = x1.groupby(by = ['PostId', 'VoteDate'])
x1 = tmp.size().reset_index()
x1.columns = ['Id', 'VoteDate', 'Total']


x1['NewVotes'] = x1['VoteDate'] * x1['Total']
x1['OldVotes'] = (1 - x1['VoteDate']) * x1['Total']
tmp = x1.groupby(by = 'Id')

x = pd.merge(x1, tmp['OldVotes'].max().reset_index(), how = 'inner', on = 'Id')
x = pd.merge(x, tmp['NewVotes'].max().reset_index(), how = 'inner', on = 'Id')
x = pd.merge(x, tmp['Total'].sum().reset_index(), how = 'inner', on = 'Id')

x = x[['Id', 'OldVotes_y', 'NewVotes_y']]
x.columns = ['Id', 'OldVotes', 'NewVotes']

x = x[x.NewVotes == 0]
x = pd.merge(x, Posts, how = 'inner', on = 'Id')
x = x[x.PostTypeId == 1]
x = x[['Title', 'OldVotes']]
x = x.sort_values('OldVotes', ascending = False)
x = x.reset_index(drop = True)
x[0:10]


c:\users\staro\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,Should I avoid overnight flights from Dubai to...,23
7,"Can a visitor who is ""of age"" in their country...",23
8,What is the business model of commercial free ...,23
9,Carrying medicines internationally for a friend,23
